<a href="https://colab.research.google.com/github/Ishaqgopang/video_audio/blob/main/video_input.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -Uq google-genai

In [6]:
!pip install gTTS -q

In [7]:
!pip install -Uq playsound

  Preparing metadata (setup.py) ... done


In [16]:
from google.colab import userdata
GEMINI_API_KEY:str = userdata.get('GOOGLE_API_KEY')

In [17]:
from google import genai
from google.genai import Client

client: Client = genai.Client(
    api_key=GEMINI_API_KEY,
)

model: str = "gemini-2.0-flash-exp"

In [18]:
video_file = "/content/Introductory.mp4"

In [19]:
import time

def upload_video(video_file):
  video_file = client.files.upload(path = video_file)
  while video_file.state == "Processing":
    print('waiting for video to be processed')
    time.sleep(10)
    video_file = client.files.get(name = video_file.name or "")
  if video_file.state == "Failed":
    raise ValueError(video_file.state)
  print(f'video processing complete.' + (video_file.uri or ""))
  return video_file
my_video = upload_video(video_file)

video processing complete.https://generativelanguage.googleapis.com/v1beta/files/6k49w32jqypf


In [20]:
from google.genai.types import Content, Part
prompt = """For each scene in this video,
            generate captions that describe the scene along with any spoken text placed in quotation marks.
            Place each caption into an object with the timecode of the caption in the video.
         """

video = my_video

response = client.models.generate_content(
    model=model,
    contents=[
        Content(
            role="user",
            parts=[
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""),
                ]),
        prompt,
    ]
)

print(response.text)

```json
[
  {
    "timecode": "00:00",
    "caption": "A person is sitting in front of a computer screen wearing a blue jacket and light colored pants. They are speaking, and other items such as a bottle, eyeglasses, and a cup are on the table."
  },
 {
    "timecode": "00:01",
    "caption": "A person is sitting in front of a computer screen wearing a blue jacket and light colored pants. They are speaking, and other items such as a bottle, eyeglasses, and a cup are on the table. They say, \"Assalam-o-alaikum, mera naam Mohammed Ishtiaq hai, aur main PIA-IC mein batch 16 2 mein part two ka student hun, Ajitech AI ka.\""
 }
]
```


In [27]:
from gtts import gTTS
from playsound import playsound
from IPython.display import Audio

tts = gTTS(text = "\A person is sitting in front of a computer screen wearing a blue jacket and light colored pants. They are speaking, and other items such as a bottle, eyeglasses, and a cup are on the table. They say, \"Assalam-o-alaikum, mera naam Mohammed Ishtiaq hai, aur main PIA-IC mein batch 16 2 mein part two ka student hun, Ajitech AI ka."\
,lang= 'en')
with open("output.mp3", "wb") as f:
  for chunk in tts.stream():
    f.write(chunk)
display(Audio("output.mp3", autoplay=True))